In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('files/train.csv')
test = pd.read_csv('files/test.csv')

df = pd.concat([train,test])

In [3]:
df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,2919.000000,2919.000000,2433.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2896.000000,2918.000000,...,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,1460.000000
mean,1460.000000,57.137718,69.305795,10168.114080,6.089072,5.564577,1971.312778,1984.264474,102.201312,441.423235,...,93.709832,47.486811,23.098321,2.602261,16.062350,2.251799,50.825968,6.213087,2007.792737,180921.195890
std,842.787043,42.517628,23.344905,7886.996359,1.409947,1.113131,30.291442,20.894344,179.334253,455.610826,...,126.526589,67.575493,64.244246,25.188169,56.184365,35.663946,567.402211,2.714762,1.314964,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,730.500000,20.000000,59.000000,7478.000000,5.000000,5.000000,1953.500000,1965.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129975.000000
50%,1460.000000,50.000000,68.000000,9453.000000,6.000000,5.000000,1973.000000,1993.000000,0.000000,368.500000,...,0.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,2189.500000,70.000000,80.000000,11570.000000,7.000000,6.000000,2001.000000,2004.000000,164.000000,733.000000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,2919.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,1424.000000,742.000000,1012.000000,508.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000,755000.000000


# Check for NaN columns..


In [4]:
for cols in df.columns:
    if df[cols].isnull().any():
        print(cols)

MSZoning
LotFrontage
Alley
Utilities
Exterior1st
Exterior2nd
MasVnrType
MasVnrArea
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinSF1
BsmtFinType2
BsmtFinSF2
BsmtUnfSF
TotalBsmtSF
Electrical
BsmtFullBath
BsmtHalfBath
KitchenQual
Functional
FireplaceQu
GarageType
GarageYrBlt
GarageFinish
GarageCars
GarageArea
GarageQual
GarageCond
PoolQC
Fence
MiscFeature
SaleType
SalePrice


# Let's go through how I might want to treat each column, before I get into dealing with NaN's...

## MSSubClass

The values it takes are kind of weird considering its categorical data. If I decide to use a decision tree-based algorithm then it's fine. If not I'll need to at least normailize this one.

## MSZoning 
Should be one-hot encoded.

## LotFrontage, LotArea
Numerical data. We're dealing with a bunch of different units here, and if I wasn't working with a decision tree I'd really need to be careful to normalize and standardize the data.

## Street, Alley, LandContour, LotConfig, Neighborhood, Condition1, Condition2, BldgType, HouseStyle
Categorical. To be one-hot encoded.

## OverallCond and OverallQual
Numerical.

## YearBuilt, YearRemodAdd
Numerical data. Need to think about how I should treat this. Could maybe base them off of how many days ago they were built / remodeled.

## RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType
All categorical.

## MasVnrArea
Numerical.

## LotShape, Utilities, LandSlope, ExterQual, ExterCond
Categorical, but admits an ordered set, with Excellent > Good > Average/Typical etc.. so can convert to numerical to save on columns

## Foundation
Categorical.

## BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1
Categorical, but again admitting an ordered set, so should be converted into numerical data to save on columns.

## BsmtFinSF1
Numerical.

## BsmtFinType2
Categorical, but admitting an ordered set -- convert to numeric.

## BsmtFinSF2, BsmtUnfSF, TotalBsmtSF
Numerical.

## Heating
Categorical.

## HeatingQC
Categorical, but ordered set so convert to numeric.

## CentralAir, Electrical
Categorical.

## 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, BsmtFullBath,  BsmtHalfBath, FullBath, HalfBath, Bedroom, Kitchen
Numerical

## KitchenQual
Categoric, but admits ordered set, so convert to numerical.

## TotRmsAbvGrd
Numerical.

## Functional
Categorical, but admits ordered set. Convert to numerical.

## Fireplaces
Numerical.

## FireplaceQu
Categorical -- convertable to numerical.

## GarageType
Categorical.

## GarageFinish
Categorical -- convertable to numerical.

## GarageYrBlt
Possibly convert to 'how many days ago'.

## GarageCars, GarageArea
Numerical.

## GarageQual, GarageCond
Categorical -- convertable to numerical.

## PavedDrive
I think this admits an ordered set in terms of 'paved-ness'. So possibly convertable to numerical, although could be safe and just keep it categorical -- only like 3 unique values.

## WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea
Numerical.

## PoolQC, Fence
Categorical -- convertable to numerical.

## MiscFeature
Categorical.

## MiscVal
Numerical.

## MoSold , YrSold
MoSold might not be worth the effort to include. I won't want to one-hot encode 12 columns for each month, and I doubt the month something was sold can be worth the 12 columns I'd be costing the training data. YrSold I can express in terms of 'days since the present'. I could maybe express MoSold as numerical data of values between 1 and 12. But I don't think that's justifiable because it's weird to say that month 2 > month 1 or something.

## SaleType, SaleCondition
Categorical.

# Okay, with that, here are my next steps:

There isn't a whole lot of rows to this data, so if I add too many columns we start worrying about big-p, little-n issues. I need to try and impute my NaN's instead of deleting those rows so that the little-n doesn't get even smaller. I'll also really want to favor converting categorical data that admits an ordered set into numerical data so I can save up on columns.
1. Create a function to deal with missing NaN values sample-by-sample by selecting a subset of the data that shares as many characteristics as possible with a sample and picking the most likely value to impute given this. 
2. Create a function that converts categorical-but-numerical-convertable columns into numeric columns.
3. Deal with time-series columns.

It might be worth trying to find similar columns by one-hot encoding and normalizing data, and seeing which samples have the highest cosine similarity to the one in question.. I'd need to normalize numerical data, or else different magnitudes would have disproportionate influence on the the cosine similarity..

# One-hot encoding categorical data

Also going to drop the MoSold column.

In [5]:
df = df.drop(columns = ['MoSold'])

In [6]:
to_one_hot = [
    'MiscFeature',
    'SaleType',
    'SaleCondition',
    'CentralAir', 
    'Electrical',
    'Heating',
    'Foundation',
    'RoofStyle', 
    'RoofMatl', 
    'Exterior1st', 
    'Exterior2nd', 
    'MasVnrType',
    'Street', 
    'Alley', 
    'LandContour', 
    'LotConfig', 
    'Neighborhood', 
    'Condition1', 
    'Condition2', 
    'BldgType', 
    'HouseStyle',
    'MSZoning',
    'MSSubClass',
    'GarageType',
    'PavedDrive'
    ]

In [7]:
time_series_columns = ['YrSold','YearBuilt','YearRemodAdd','GarageYrBlt']

And now for the data I want to make numeric.

In [8]:
numerical = [
    'MiscVal',
    'WoodDeckSF', 
    'OpenPorchSF', 
    'EnclosedPorch', 
    '3SsnPorch', 
    'ScreenPorch', 
    'PoolArea',
    'GarageCars', 
    'GarageArea',
    'Fireplaces',
    'TotRmsAbvGrd',
    '1stFlrSF', 
    '2ndFlrSF', 
    'LowQualFinSF', 
    'GrLivArea', 
    'BsmtFullBath',  
    'BsmtHalfBath', 
    'FullBath', 
    'HalfBath', 
    'BedroomAbvGr', 
    'KitchenAbvGr',
    'BsmtFinSF2', 
    'BsmtUnfSF', 
    'TotalBsmtSF',
    'BsmtFinSF1',
    'MasVnrArea',
    'LotFrontage', 
    'LotArea',
    'OverallCond',
    'OverallQual',
      
]

In [9]:
columns_so_far = to_one_hot + time_series_columns + numerical

In [10]:
full_columns = list(df.columns)

In [11]:
to_numerical = list(set(columns_so_far).symmetric_difference(full_columns))

In [12]:
to_numerical.remove("Id")

In [13]:
to_numerical.remove("SalePrice")

## One-hot encode the categorical columns

In [14]:
df = pd.get_dummies(df,columns = to_one_hot)

## Convert the to_numerical data to numerical

In [15]:
import json 
with open('files/to_numerical_json.json', 'r') as f:   
    ordered_categories = json.load(f)

In [16]:
ordered_categories

{'Fence': ['GdPrv', 'MnPrv', 'GdWo', 'MnWw', 'NA'],
 'LotShape': ['Reg', 'IR1', 'IR2', 'IR3'],
 'Utilities': ['AllPub', 'NoSewr', 'NoSeWa', 'ELO'],
 'LandSlope': ['Gtl', 'Mod', 'Sev'],
 'BsmtQual': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
 'BsmtCond': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
 'BsmtExposure': ['Gd', 'Av', 'Mn', 'No', 'NA'],
 'BsmtFinType1': ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA'],
 'BsmtFinType2': ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA'],
 'HeatingQC': ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
 'KitchenQual': ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
 'Functional': ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal'],
 'FireplaceQu': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
 'GarageFinish': ['Fin', 'RFn', 'Unf', 'NA'],
 'ExterCond': ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
 'ExterQual': ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
 'GarageQual': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
 'GarageCond': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
 'PoolQC': ['Ex', 'Gd', 'TA', 'Fa', 'NA']}

In [17]:
from funcs.conv_to_numerical import conv_to_numerical

In [18]:
df = conv_to_numerical(df, ordered_categories)

## Convert dates into 'days ago'
Going to assume all houses listed for a given year were listed on January 1st of that year. Because I'm omitting month, the time during the year won't matter.

In [19]:
from funcs.get_days_ago import get_days_ago

In [20]:
for cols in time_series_columns:
    df[cols] = [get_days_ago(year) for year in df[cols]]

# Normalize the numerical columns for the cosine similarity steps
Normalizing is not generally a good idea for a decision tree-based model, so I'll probably make it separate to df. I'll also add the time-series columns to the numerical data now that I've converted it to numerical data.

In [21]:
all_numerical = to_numerical + numerical + time_series_columns

In [22]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
#cos_sim_df = cos_sim_df.drop(columns = numerical)
df[all_numerical] = min_max_scaler.fit_transform(df[all_numerical])

## Now to implement the cosine similarity function to find a subset of data most similar to a sample with one or more NaN values. Just need to find the cosine similarity between the sample with NaN of interest with all other columns, dealing with NaN's in the cosine similarity (either make the similarity zero or NaN or something) and get a subset of the data which has a cosine similarity score above a certain threshold. From that subset, take the mean/median of the values of the subset to fill in the NaN for that sample.

In [23]:
from funcs.get_most_similar_rows import get_most_similar_rows

results_train = get_most_similar_rows(df.iloc[:train.shape[0]], 'train')


In [24]:
results_test = get_most_similar_rows(df.iloc[train.shape[0]:], 'test')

In [25]:
df.isnull().sum(axis = 1).sum()


7678

In [26]:
# In order to get the 'best' subset, take the top nth (maybe 70th?) percentile of cos_sim scores as the subset. Then, just form a subset of df 
# with those indices that pass.
from funcs.get_most_similar_rows import smart_fillna

cutoff = 0.75 # Could use GridsearchCV to find best cutoff?

df.iloc[:train.shape[0]] = smart_fillna(df.iloc[:train.shape[0]], cutoff, results_train, all_numerical) 


/Users/sangersteel/miniforge3/envs/toxic/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/sangersteel/miniforge3/envs/toxic/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/Users/sangersteel/Downloads/house-prices-advanced-regression-techniques/funcs/get_most_similar_rows.py:96: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(subset_test)
/Users/sangersteel/miniforge3/envs/toxic/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  wa

In [27]:
df.iloc[train.shape[0]:] = smart_fillna(df.iloc[train.shape[0]:], cutoff, results_test, all_numerical) 


/Users/sangersteel/Downloads/house-prices-advanced-regression-techniques/funcs/get_most_similar_rows.py:99: RuntimeWarning: invalid value encountered in double_scalars
  if np.abs((mean-median)/mean)*100 < 10: # Mean and median are close together -- can pick either, will pick mean


In [28]:
df.isnull().sum(axis = 1).sum()


1587

In [29]:
df = df.drop(columns = ['Id'])

In [30]:
df.isnull().sum(axis = 1).sum()

1587

In [31]:
train = df.iloc[:train.shape[0]]
test = df.iloc[train.shape[0]:]

In [32]:
train = train.sample(frac = 1)

In [33]:
test

,LotFrontage,LotArea,LotShape,Utilities,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,...,MSSubClass_190,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,PavedDrive_N,PavedDrive_P,PavedDrive_Y
0,0.202055,0.048246,1.000000,0.0,1.0,0.444444,0.625,0.355072,0.816667,0.00000,...,0,0,1,0,0,0,0,0,0,1
1,0.205479,0.060609,0.666667,0.0,1.0,0.555556,0.625,0.376812,0.866667,0.06750,...,0,0,1,0,0,0,0,0,0,1
2,0.181507,0.058566,0.666667,0.0,1.0,0.444444,0.500,0.094203,0.200000,0.00000,...,0,0,1,0,0,0,0,0,0,1
3,0.195205,0.040562,0.666667,0.0,1.0,0.555556,0.625,0.086957,0.200000,0.01250,...,0,0,1,0,0,0,0,0,0,1
4,0.075342,0.017318,0.666667,0.0,1.0,0.777778,0.500,0.130435,0.300000,0.00000,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,0.000000,0.002973,1.000000,0.0,1.0,0.333333,0.750,0.289855,0.666667,0.00000,...,0,0,0,0,0,0,0,0,0,1
1455,0.000000,0.002776,1.000000,0.0,1.0,0.333333,0.500,0.289855,0.666667,0.00000,...,0,0,0,0,0,1,0,0,0,1
1456,0.476027,0.087406,1.000000,0.0,1.0,0.444444,0.750,0.362319,0.233333,0.00000,...,0,0,0,0,0,0,1,0,0,1
1457,0.140411,0.042726,1.000000,0.0,1.0,0.444444,0.500,0.130435,0.300000,0.00000,...,0,0,0,0,0,0,0,0,0,1


In [34]:
train

,LotFrontage,LotArea,LotShape,Utilities,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,...,MSSubClass_190,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,PavedDrive_N,PavedDrive_P,PavedDrive_Y
68,0.089041,0.015462,0.666667,0.0,1.0,0.333333,0.625,0.471014,1.000000,0.000000,...,0,0,1,0,0,0,0,0,0,1
1427,0.133562,0.045012,1.000000,0.0,1.0,0.444444,0.625,0.471014,1.000000,0.000000,...,0,0,1,0,0,0,0,0,0,1
1110,0.184932,0.031316,0.333333,0.0,1.0,0.555556,0.500,0.108696,0.233333,0.000000,...,0,0,1,0,0,0,0,0,0,1
229,0.075342,0.008797,0.666667,0.0,1.0,0.666667,0.500,0.036232,0.066667,0.010000,...,0,0,1,0,0,0,0,0,0,1
957,0.167808,0.028605,1.000000,0.0,1.0,0.444444,0.500,0.347826,0.800000,0.000000,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,0.551370,0.062035,0.666667,0.0,1.0,0.666667,0.500,0.043478,0.100000,0.000000,...,0,0,1,0,0,0,0,0,0,1
589,0.099315,0.036458,0.666667,0.0,1.0,0.444444,0.625,0.579710,0.833333,0.000000,...,0,0,0,0,0,0,1,0,1,0
6,0.184932,0.041057,0.666667,0.0,1.0,0.777778,0.500,0.043478,0.083333,0.116250,...,0,0,1,0,0,0,0,0,0,1
281,0.133562,0.027577,1.000000,0.0,1.0,0.555556,0.500,0.028986,0.066667,0.042500,...,0,0,1,0,0,0,0,0,0,1


In [35]:
train = train.dropna()

In [36]:
y= train['SalePrice']
x = train.drop(columns = ['SalePrice'])

# Now to finally try a model fitting

In [37]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2)

In [38]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
grid_param= {
                 'max_depth' : [4, 6, 8],
                 'n_estimators': [50, 10],
                 'max_features': ['sqrt', 'auto', 'log2'],
                 'min_samples_split': [2, 3, 10],
                 'min_samples_leaf': [1, 3, 10],
                 'bootstrap': [True, False],
}

In [39]:
rfg = RandomForestRegressor()
rfg_grid = GridSearchCV(estimator = rfg,  param_grid = grid_param, cv = 5)
rfg_grid.fit(x_train, y_train)


Fitting 5 folds for each of 324 candidates, totalling 1620 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'bootstrap': [True, False], 'max_depth': [4, 6, 8],
                         'max_features': ['sqrt', 'auto', 'log2'],
                         'min_samples_leaf': [1, 3, 10],
                         'min_samples_split': [2, 3, 10],
                         'n_estimators': [50, 10]},
             verbose=1)

In [40]:
print('Best score: {}'.format(rfg_grid.best_score_))
print('Best parameters: {}'.format(rfg_grid.best_params_))
best_params = rfg_grid.best_params_

Best score: 0.8448231592830018
Best parameters: {'bootstrap': True, 'max_depth': 8, 'max_features': 'auto', 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 50}


In [41]:
rfg = RandomForestRegressor(**best_params)

In [42]:
rfg.fit(x_train, y_train)

RandomForestRegressor(max_depth=8, min_samples_leaf=3, min_samples_split=10,
                      n_estimators=50)

In [43]:
rfg.score(x_test, y_test)

0.8536445775577332

In [44]:
test = test.drop(columns = ['SalePrice', ])

Sometimes, even the whole subset has NaN for the value I'm trying to fill in. I can impute zero and just deal with it.

In [45]:
test = test.fillna(0) 

In [46]:
predictions = rfg.predict(test)

In [47]:
test['Id'] = [x + 1461 for x in list(test.index)]

In [48]:
test['SalePrice'] = predictions

In [49]:
test[["Id","SalePrice"]].to_csv('out.csv',index = False)